In [ ]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib
import matplotlib.ticker
import matplotlib.pyplot as plt

import pathlib

from typing import Optional, List
from scipy import stats
from seaborn_box_width_fix import adjust_box_widths

%matplotlib inline

In [ ]:
VALID_ENVIRONMENTS = ["local", "kind", "same-pod", "same-cluster"]

ENVIRONMENT_NAMING = {
    "local": "in Docker",
    "kind": "in Kind",
    "same-pod": "in EKS pod",
    "same-cluster": "across EKS nodes",
}

HOOK_NAMING = {
    True: "With function hook",
    False: "Original",
}

# fmt: off
FILE_MAPPING = {
    ("local", True): "docker-with-benchmark_15-35-50_1691768150270_8081.csv",
    ("local", False): "docker-without-hook-benchmark_15-30-06_1691767806057.csv",
    ("kind", True): "kind-with-hook-benchmark_16-30-45_1691771445417.csv",
    ("kind", False): "kind-without-hook-benchmark_16-25-22_1691771122967.csv",
    ("same-pod", True): "aws-pod-with-hook-benchmark_11-31-08_1691753468729.csv",
    ("same-pod", False): "aws-pod-without-hook-benchmark_09-24-46_1691745886638.csv",
    ("same-cluster", True): "aws-nodes-with-hook-benchmark_11-46-54_1691754414125.csv",
    ("same-cluster", False): "aws-nodes-without-hook-benchmark_11-40-50_1691754050578.csv",
}
# fmt: on

RC_CONTEXT = {
    "text.usetex": True,
    "font.family": "serif",
    "font.serif": "Computer Modern",
    "font.size": 20,
    "savefig.bbox": "tight",
    "savefig.pad_inches": 0.0,
}


def generate_vector(env: str, hook: bool, ncycles: int) -> np.ndarray:
    """Generates mocked latency values with warm-up, for a given setup"""

    assert env in VALID_ENVIRONMENTS
    mu = {"local": 3, "kind": 3.5, "same-pod": 1.5, "same-cluster": 2.2}
    std = {"local": 0.1, "kind": 0.2, "same-pod": 0.4, "same-cluster": 0.75}
    rng = np.random.Generator(np.random.PCG64())
    x = rng.normal(mu[env] + (0.5 if hook else 0), std[env], ncycles)

    # add exponential decay to the beginning of the vector
    warmup_strength, warmup_ratio = 0.7, 0.1
    warmup = np.linspace(0, 1, int(ncycles * warmup_ratio))
    warmup = 1.0 + np.exp(-5 * warmup) * (warmup_strength / std[env])
    x[: len(warmup)] *= warmup

    return x


def load_dataframe(basepath: str) -> pd.DataFrame:
    """Load all dataframes from CSV files in a given path"""
    basepath = pathlib.Path(basepath)
    dfs = []

    for (env, hook), filename in FILE_MAPPING.items():
        vec = pd.read_csv(basepath.joinpath(filename))["response_time"]
        print(
            f"read {filename} for ({env}, {hook}) [{len(vec)=}, {vec.mean()=}, {vec.median()=}]"
        )

        vec_df = pd.DataFrame.from_dict({"time": vec, "env": env, "hook": hook})
        vec_df.reset_index(inplace=True)
        vec_df.rename(columns={"index": "iteration"}, inplace=True)
        dfs.append(vec_df)

    df = pd.concat(dfs, ignore_index=True)
    return df


def generate_dataframe(ncycles: int = 1_000) -> pd.DataFrame:
    """Generate a fake dataframe in wide format"""
    dfs = []

    for env in VALID_ENVIRONMENTS:
        for hook in [True, False]:
            vec = generate_vector(env, hook, ncycles=ncycles)
            vec_df = pd.DataFrame.from_dict({"time": vec, "env": env, "hook": hook})
            vec_df.reset_index(inplace=True)
            vec_df.rename(columns={"index": "iteration"}, inplace=True)
            dfs.append(vec_df)

    df = pd.concat(dfs, ignore_index=True)
    return df


def filter_dataframe(
    df: pd.DataFrame, start: Optional[int] = None, stop: Optional[int] = None
) -> pd.DataFrame:
    """Per condition, remove the first n values"""
    start = start or df.iteration.min()
    stop = stop or df.iteration.max()
    print(f"\\newcommand{{\\TestTotalSampleSize}}{{{df.iteration.nunique():,d}}}")
    print(f"\\newcommand{{\\TestWarmUpPeriodSize}}{{{start:,d}}}")
    return df.query("iteration >= @start and iteration <= @stop")


def test_dataframe(df: pd.DataFrame):
    """Per condition, perform a t-test for difference in means"""
    result = []
    sample_size = None
    for env in VALID_ENVIRONMENTS:
        env_df = df.query("env == @env")
        with_hook_df = env_df.query("hook == True").time
        without_hook_df = env_df.query("hook == False").time

        sample_size = sample_size or len(with_hook_df)
        assert sample_size == len(with_hook_df) == len(without_hook_df)

        n1, n2 = len(with_hook_df), len(without_hook_df)
        s1, s2 = with_hook_df.std(), without_hook_df.std()
        m1, m2 = with_hook_df.mean(), without_hook_df.mean()

        # pooled standard deviation (computed for equal and unequal sample sizes)
        sp_equal_n = np.sqrt((s1**2 + s2**2) / 2)
        sp_unequal_n = np.sqrt(
            ((n1 - 1) * s1**2 + (n2 - 1) * s2**2) / (n1 + n2 - 2)
        )
        assert np.isclose(sp_equal_n, sp_unequal_n), f"{sp_equal_n=} != {sp_unequal_n=}"

        test = stats.ttest_ind(with_hook_df, without_hook_df)
        result.append(
            {
                "env": env,
                "p-value": test.pvalue,
                "statistic": test.statistic,
                "n": sample_size,
                "sp": sp_equal_n,
            }
        )

        # sanity check that the test statistic is as in the formular from the paper
        t_paper = (m1 - m2) / (sp_equal_n * np.sqrt(2 / sample_size))
        assert np.isclose(test.statistic, t_paper), f"{test.statistic=} != {t_paper=}"

    df = pd.DataFrame.from_dict(result)
    df.set_index("env", inplace=True)

    # fmt: off
    macros = []
    macros.append(f"\\newcommand{{\\TestMeasurementPeriodSize}}{{{sample_size:,d}}}")
    macros.append(f"\\newcommand{{\\TestResultLocalPValue}}{{{df.loc['local', 'p-value']:.4f}}}")
    macros.append(f"\\newcommand{{\\TestResultLocalStdPool}}{{{df.loc['local', 'sp']:.2f}}}")
    macros.append(f"\\newcommand{{\\TestResultKindPValue}}{{{df.loc['kind', 'p-value']:.4f}}}")
    macros.append(f"\\newcommand{{\\TestResultKindStdPool}}{{{df.loc['kind', 'sp']:.2f}}}")
    macros.append(f"\\newcommand{{\\TestResultSamePodPValue}}{{{df.loc['same-pod', 'p-value']:.4f}}}")
    macros.append(f"\\newcommand{{\\TestResultSamePodStdPool}}{{{df.loc['same-pod', 'sp']:.2f}}}")
    macros.append(f"\\newcommand{{\\TestResultSameClusterPValue}}{{{df.loc['same-cluster', 'p-value']:.4f}}}")
    macros.append(f"\\newcommand{{\\TestResultSameClusterStdPool}}{{{df.loc['same-cluster', 'sp']:.2f}}}")
    # fmt: on

    print("\n".join(macros))
    display(df)


@matplotlib.rc_context(RC_CONTEXT)
def plot_boxplots(df: pd.DataFrame):
    sns.set_theme(style="whitegrid", palette="pastel")
    fig = plt.figure(figsize=(5.5, 2.5))

    df = df.copy(deep=True)
    df["hook"] = df["hook"].map(HOOK_NAMING)
    df["env"] = df["env"].map(ENVIRONMENT_NAMING)

    ax = sns.boxplot(
        data=df,
        x="env",
        y="time",
        hue="hook",
        hue_order=[HOOK_NAMING[False], HOOK_NAMING[True]],
        palette=["#b2df8a", "#a6cee3"],
        width=0.75,
        showcaps=True,
        showfliers=False,
        linewidth=0.5,
    )

    sns.despine(offset=10, trim=False)
    ax.set_ylabel("RTT (ms)")
    ax.set_xlabel("")
    ax.legend(loc="lower left", frameon=False, fancybox=False, ncol=2)
    ax.legend_.set_bbox_to_anchor((0, -0.1, 1, 1))
    ax.yaxis.set_major_locator(matplotlib.ticker.MultipleLocator(1))
    ax.yaxis.grid(True, ls=(0, (1, 3)), which="major", color="grey", alpha=0.25, lw=1)
    ax.set_ylim(0, 5)

    adjust_box_widths(fig, 0.8)

    plt.tight_layout()
    plt.savefig("boxplots.pdf")
    plt.show()


@matplotlib.rc_context(RC_CONTEXT)
def plot_lagplots(
    df: pd.DataFrame,
    envs: List[List[str]],
    cutoff: int,
    filename: str = "lagplots.pdf",
):
    colors = ["#66c2a5", "#fc8d62", "#8da0cb", "#e78ac3"]
    sns.set_theme(style="whitegrid", palette="pastel")
    fig, axes = plt.subplots(len(envs), 1, figsize=(5.5, 2.5), sharex=True)

    print(f"\\newcommand{{\\TestLagplotCutoff}}{{{cutoff:,d}}}")

    df = df.query("hook == False and iteration < @cutoff").copy(deep=True)
    df["hook"] = df["hook"].map(HOOK_NAMING)
    df["env"] = df["env"].map(ENVIRONMENT_NAMING)

    for env, ax in zip(envs, axes):
        sns.lineplot(
            ax=ax,
            data=df,
            y="time",
            x="iteration",
            hue="env",
            hue_order=[ENVIRONMENT_NAMING[e] for e in env],
            linewidth=0.25,
            alpha=0.75,
            palette=colors[1:3],
        )

        sns.despine(ax=ax, offset=10, trim=False)
        ax.set_ylabel("RTT (ms)", fontsize=10)
        ax.set_xlabel("Iterations")
        ax.legend(loc="lower left", frameon=False, fancybox=False, ncol=2, fontsize=10)
        ax.legend_.set_bbox_to_anchor((0, -0.4, 1, 1))
        ax.yaxis.grid(
            True, ls=(0, (1, 3)), which="both", color="grey", alpha=0.25, lw=1
        )
        ax.set_yscale("log")
        ax.set_ylim(1, 20)
        ax.xaxis.grid(False)

        tickform = matplotlib.ticker.FuncFormatter(lambda x, _: format(int(x), ","))
        ax.xaxis.set_major_formatter(tickform)
        ax.yaxis.set_major_formatter(tickform)

    # fig.text(0, 0.6, "Latency (ms)", va="center", rotation="vertical")

    fig.tight_layout()
    fig.savefig(filename)
    plt.show()


df = load_dataframe("./data/paper")
# df = generate_dataframe(ncycles=50_000)
dff = filter_dataframe(df, start=4_000)

test_dataframe(dff)
plot_boxplots(dff)
plot_lagplots(df, envs=[["kind", "local"], ["same-cluster", "same-pod"]], cutoff=10_000)
